In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.palettes import brewer
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.models.ranges import Range1d
output_notebook()

data = pd.read_excel('data.xlsx')
data.head()

Loading BokehJS ...

,city,cost_of_living_index,air_pollution,av_temp,global_cities_index
0,Austin,144,9.000000,20.3,40
1,Bali,83,33.000000,27.0,56
2,Barcelona,143,15.029966,15.5,24
3,Beirut,126,32.000000,20.0,99
4,Berlin,147,16.293344,9.0,14


In [2]:
# adjusting gci rank for the sake of better visualisation
# flipping the index, so the highest rank have the highest index number
data['gci_adjusted'] = ((data['global_cities_index'] - 100)/(3))*(-1) 
# artificially bringing up Beirut's index number so it can be seen on the graph more easily
data.iloc[14,5] = 5

In [3]:
source = ColumnDataSource(data)

# set colour of plot points to represent temperature
colour_mapper = LinearColorMapper(
        palette='Viridis256',
        low=data["av_temp"].min(),
        high=data["av_temp"].max())

# set info on mouse hover
hover = HoverTool(tooltips=[('/c/','@city'),
                            ('avg. annual temp (°C)', '@av_temp'), 
                            ('global cities index', '@global_cities_index'),
                            ('cost of living index', '@cost_of_living_index')])

# setting up plot object
p = figure(plot_width=700,
           plot_height=500,
           title="Livable Cities",
           tools=[hover],
           min_border=1,
           x_axis_label='Avg. Annual Air Pollution (PM2.5)',
           y_axis_label='Cost of Living Index (100 = Prague)'
          )

# plot chart points
p.circle('air_pollution', 
         'cost_of_living_index', 
         size='gci_adjusted', 
         fill_alpha=0.5, 
         source=source, 
         color={'field': 'av_temp', 'transform': colour_mapper}
        )

# adding colour bar legend to the right of plot to show temperature
color_bar = ColorBar(color_mapper=colour_mapper,
                     label_standoff=12, border_line_color=None, 
                     location=(0,0), title='°C')
p.add_layout(color_bar, 'right')

# clean up the fig space
p.grid.visible = False

p.toolbar.logo = None
p.toolbar_location = None

p.xaxis.major_tick_line_color = None  
p.xaxis.minor_tick_line_color = None  

p.yaxis.major_tick_line_color = None  
p.yaxis.minor_tick_line_color = None


show(p)